In [1]:
!rm -r -f /content/sample_data/Results
!rm -r -f /content/sample_data/REFS
!rm -r -f /content/sample_data/TARGET

!mkdir  /content/sample_data/REFS
!mkdir  /content/sample_data/Results
!mkdir  /content/sample_data/TARGET



In [2]:
#@title
import moviepy.video.fx.all as vfx
from moviepy.editor import VideoFileClip

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from sklearn.cluster import KMeans
import math




from PIL import Image
import numpy as np
from skimage import color
import torch
import torch.nn.functional as F
from IPython import embed
import torch
import torch.nn as nn
import numpy as np
from IPython import embed

from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx
def load_img(img_path):
	out_np = np.asarray(Image.open(img_path))
	if(out_np.ndim==2):
		out_np = np.tile(out_np[:,:,None],3)
	return out_np

def resize_img(img, HW=(256,256), resample=3):
	return np.asarray(Image.fromarray(img).resize((HW[1],HW[0]), resample=resample))

def preprocess_img(img_rgb_orig, HW=(256,256), resample=3):
	# return original size L and resized L as torch Tensors
	img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)

	img_lab_orig = color.rgb2lab(img_rgb_orig)
	img_lab_rs = color.rgb2lab(img_rgb_rs)

	img_l_orig = img_lab_orig[:,:,0]
	img_l_rs = img_lab_rs[:,:,0]

	tens_orig_l = torch.Tensor(img_l_orig)[None,None,:,:]
	tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]

	return (tens_orig_l, tens_rs_l)

def postprocess_tens(tens_orig_l, out_ab, mode='bilinear'):
	# tens_orig_l 	1 x 1 x H_orig x W_orig
	# out_ab 		1 x 2 x H x W

	HW_orig = tens_orig_l.shape[2:]
	HW = out_ab.shape[2:]

	# call resize function if needed
	if(HW_orig[0]!=HW[0] or HW_orig[1]!=HW[1]):
		out_ab_orig = F.interpolate(out_ab, size=HW_orig, mode='bilinear')
	else:
		out_ab_orig = out_ab

	out_lab_orig = torch.cat((tens_orig_l, out_ab_orig), dim=1)
	return color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))

import torch
from torch import nn

class BaseColor(nn.Module):
	def __init__(self):
		super(BaseColor, self).__init__()

		self.l_cent = 50.
		self.l_norm = 100.
		self.ab_norm = 110.

	def normalize_l(self, in_l):
		return (in_l-self.l_cent)/self.l_norm

	def unnormalize_l(self, in_l):
		return in_l*self.l_norm + self.l_cent

	def normalize_ab(self, in_ab):
		return in_ab/self.ab_norm

	def unnormalize_ab(self, in_ab):
		return in_ab*self.ab_norm





class ECCVGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d):
        super(ECCVGenerator, self).__init__()

        model1=[nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]

        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]

        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]

        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        model8=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]

        model8+=[nn.Conv2d(256, 313, kernel_size=1, stride=1, padding=0, bias=True),]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8 = nn.Sequential(*model8)

        self.softmax = nn.Softmax(dim=1)
        self.model_out = nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=False)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear')

    def forward(self, input_l):
        conv1_2 = self.model1(self.normalize_l(input_l))
        conv2_2 = self.model2(conv1_2)
        conv3_3 = self.model3(conv2_2)
        conv4_3 = self.model4(conv3_3)
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)
        conv8_3 = self.model8(conv7_3)
        out_reg = self.model_out(self.softmax(conv8_3))

        return self.unnormalize_ab(self.upsample4(out_reg))

def eccv16(pretrained=True):
	model = ECCVGenerator()
	if(pretrained):
		import torch.utils.model_zoo as model_zoo
		model.load_state_dict(model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/colorization_release_v2-9b330a0b.pth',map_location='cpu',check_hash=True))
	return model

class SIGGRAPHGenerator(BaseColor):
    def __init__(self, norm_layer=nn.BatchNorm2d, classes=529):
        super(SIGGRAPHGenerator, self).__init__()

        # Conv1
        model1=[nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True),]
        model1+=[nn.ReLU(True),]
        model1+=[norm_layer(64),]
        # add a subsampling operation

        # Conv2
        model2=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model2+=[nn.ReLU(True),]
        model2+=[norm_layer(128),]
        # add a subsampling layer operation

        # Conv3
        model3=[nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model3+=[nn.ReLU(True),]
        model3+=[norm_layer(256),]
        # add a subsampling layer operation

        # Conv4
        model4=[nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model4+=[nn.ReLU(True),]
        model4+=[norm_layer(512),]

        # Conv5
        model5=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model5+=[nn.ReLU(True),]
        model5+=[norm_layer(512),]

        # Conv6
        model6=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[nn.Conv2d(512, 512, kernel_size=3, dilation=2, stride=1, padding=2, bias=True),]
        model6+=[nn.ReLU(True),]
        model6+=[norm_layer(512),]

        # Conv7
        model7=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True),]
        model7+=[nn.ReLU(True),]
        model7+=[norm_layer(512),]

        # Conv7
        model8up=[nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=True)]
        model3short8=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]

        model8=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True),]
        model8+=[nn.ReLU(True),]
        model8+=[norm_layer(256),]

        # Conv9
        model9up=[nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=True),]
        model2short9=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        # add the two feature maps above

        model9=[nn.ReLU(True),]
        model9+=[nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        model9+=[nn.ReLU(True),]
        model9+=[norm_layer(128),]

        # Conv10
        model10up=[nn.ConvTranspose2d(128, 128, kernel_size=4, stride=2, padding=1, bias=True),]
        model1short10=[nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),]
        # add the two feature maps above

        model10=[nn.ReLU(True),]
        model10+=[nn.Conv2d(128, 128, kernel_size=3, dilation=1, stride=1, padding=1, bias=True),]
        model10+=[nn.LeakyReLU(negative_slope=.2),]

        # classification output
        model_class=[nn.Conv2d(256, classes, kernel_size=1, padding=0, dilation=1, stride=1, bias=True),]

        # regression output
        model_out=[nn.Conv2d(128, 2, kernel_size=1, padding=0, dilation=1, stride=1, bias=True),]
        model_out+=[nn.Tanh()]

        self.model1 = nn.Sequential(*model1)
        self.model2 = nn.Sequential(*model2)
        self.model3 = nn.Sequential(*model3)
        self.model4 = nn.Sequential(*model4)
        self.model5 = nn.Sequential(*model5)
        self.model6 = nn.Sequential(*model6)
        self.model7 = nn.Sequential(*model7)
        self.model8up = nn.Sequential(*model8up)
        self.model8 = nn.Sequential(*model8)
        self.model9up = nn.Sequential(*model9up)
        self.model9 = nn.Sequential(*model9)
        self.model10up = nn.Sequential(*model10up)
        self.model10 = nn.Sequential(*model10)
        self.model3short8 = nn.Sequential(*model3short8)
        self.model2short9 = nn.Sequential(*model2short9)
        self.model1short10 = nn.Sequential(*model1short10)

        self.model_class = nn.Sequential(*model_class)
        self.model_out = nn.Sequential(*model_out)

        self.upsample4 = nn.Sequential(*[nn.Upsample(scale_factor=4, mode='bilinear'),])
        self.softmax = nn.Sequential(*[nn.Softmax(dim=1),])

    def forward(self, input_A, input_B=None, mask_B=None):
        if(input_B is None):
            input_B = torch.cat((input_A*0, input_A*0), dim=1)
        if(mask_B is None):
            mask_B = input_A*0

        conv1_2 = self.model1(torch.cat((self.normalize_l(input_A),self.normalize_ab(input_B),mask_B),dim=1))
        conv2_2 = self.model2(conv1_2[:,:,::2,::2])
        conv3_3 = self.model3(conv2_2[:,:,::2,::2])
        conv4_3 = self.model4(conv3_3[:,:,::2,::2])
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)

        conv8_up = self.model8up(conv7_3) + self.model3short8(conv3_3)
        conv8_3 = self.model8(conv8_up)
        conv9_up = self.model9up(conv8_3) + self.model2short9(conv2_2)
        conv9_3 = self.model9(conv9_up)
        conv10_up = self.model10up(conv9_3) + self.model1short10(conv1_2)
        conv10_2 = self.model10(conv10_up)
        out_reg = self.model_out(conv10_2)

        conv9_up = self.model9up(conv8_3) + self.model2short9(conv2_2)
        conv9_3 = self.model9(conv9_up)
        conv10_up = self.model10up(conv9_3) + self.model1short10(conv1_2)
        conv10_2 = self.model10(conv10_up)
        out_reg = self.model_out(conv10_2)

        return self.unnormalize_ab(out_reg)

def siggraph17(pretrained=True):
    model = SIGGRAPHGenerator()
    if(pretrained):
        import torch.utils.model_zoo as model_zoo
        model.load_state_dict(model_zoo.load_url('https://colorizers.s3.us-east-2.amazonaws.com/siggraph17-df00044c.pth',map_location='cpu',check_hash=True))
    return model
# colorizer outputs 256x256 ab map





input_video = '/content/sample_data/test2.mp4'


# Load the input video
clip = VideoFileClip(input_video)
video_duration = clip.duration
vd = math.ceil(video_duration)
height = clip.size[0]
width = clip.size[1]
cpt = 0
for it in range(0,vd,2):
    start_time =it    # Start time in seconds
    end_time =it+2   # End time in seconds

    print(start_time)
    print(end_time)
    output_video = f'/content/sample_data/TARGET/output{cpt}.mp4'

    # Crop the desired duration
    cropped_clip = clip.subclip(start_time, end_time)

    # Convert the cropped clip to grayscale
    grayscale_clip = cropped_clip.fx(vfx.blackwhite)
    grayscale_clip.ipython_display()
    # Write the grayscale clip to a new video file
    grayscale_clip.write_videofile(output_video, codec='libx264', audio_codec='aac', fps=30 )

    # Spécifiez le chemin vers le fichier h5 du modèle
    chemin_fichier = '/content/drive/MyDrive/VGG16_Grayscale (2).hdf5'

    model = tf.keras.models.load_model(chemin_fichier)

    frames = []
    framesf = []
    # Iterate over the frames and append them to the list
    for frame in grayscale_clip.iter_frames():
        plt.imshow(frame,cmap='gray')
        framef = cv2.resize(frame, (256, 256))
        framef = cv2.cvtColor(framef, cv2.COLOR_BGR2RGB)
        framef = cv2.cvtColor(framef, cv2.COLOR_RGB2GRAY)

        # Calculate the cropping coordinates

        framesf.append(framef)
        frames.append(frame)



    framesf = np.array(framesf)
    # Preprocess the frames for prediction
    framesf = framesf / 255.0
    framesf = np.expand_dims(framesf, axis=-1)




    features=model.predict(framesf)

    #SET THE K VALUE
    k = 0
    pairwise_distances = 0


    for i in range(len(features)):
        pairwise_distances = distance.euclidean(features[i], features[i-1])
        if(pairwise_distances>0.5):
            k+=1
            print(pairwise_distances)
    if (k == 0):
        k+=1

    #THE KMEANS ALGORITHM

    # Assuming 'features' is the 2D-array with shape (n_steps, feature-dimension)
    X = features

    # Set the number of clusters (K)
    num_clusters = k

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(X)
    cluster_centers = kmeans.cluster_centers_


    keyframes = []

    for center in cluster_centers:
        distances = np.linalg.norm(X - center, axis=1)
        closest_frame_index = np.argmin(distances)
        keyframes.append(closest_frame_index)

    # 'keyframes' now contains the indices of the keyframes in the original dataset

    original_video_keyframes = [frames[keyframe_index] for keyframe_index in keyframes]



    # Reshape the image data to (height, width, channels)

    # Display the Centers of the clusters "KEYFRAMES"
    plt.imshow(original_video_keyframes[0], cmap='gray')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    plt.savefig('source.png')

    #COLORIZE THE KEYFRAME
    colorizer_eccv16 = eccv16(pretrained=True).eval()
    #colorizer_siggraph17 = siggraph17(pretrained=True).eval()

    img = original_video_keyframes[0]
    (tens_l_orig, tens_l_rs) = preprocess_img(img, HW=(256,256))


    img_bw = postprocess_tens(tens_l_orig, torch.cat((0*tens_l_orig,0*tens_l_orig),dim=1))
    #out_img_siggraph17 = postprocess_tens(tens_l_orig, colorizer_siggraph17(tens_l_rs).cpu())

    out_img_eccv16 = postprocess_tens(tens_l_orig, colorizer_eccv16(tens_l_rs).cpu())


    plt.imshow(out_img_eccv16)
    print(out_img_eccv16.shape)
    plt.axis('off')
    plt.savefig(f'/content/sample_data/REFS/source{cpt}.jpg',bbox_inches='tight', pad_inches=0)
    cpt+=1


OSError: ignored

In [ ]:
%cd  /content/drive/MyDrive/
#!git clone https://github.com/zhangmozhe/Deep-Exemplar-based-Video-Colorization.git

In [ ]:
%cd  /content/drive/MyDrive/Deep-Exemplar-based-Video-Colorization/
#!git clone https://github.com/zhangmozhe/Deep-Exemplar-based-Video-Colorization.git
!ls

In [ ]:
from google.colab import files
import shutil
import os
from moviepy.editor import VideoFileClip
import os
import cv2
from IPython.display import Image
import shutil
import os


basepath = "my_test"
upload_path = os.path.join(basepath, "upload")
upload_ref_path = os.path.join(basepath, "upload_ref")
upload_output_path = os.path.join(basepath, "upload_output")

if os.path.isdir(upload_ref_path):
    shutil.rmtree(upload_ref_path)

if os.path.isdir(upload_path):
    shutil.rmtree(upload_path)

if os.path.isdir(upload_output_path):
    shutil.rmtree(upload_output_path)

os.makedirs(upload_ref_path)
os.makedirs(upload_path)
os.makedirs(upload_output_path)
for numvid in range(1,int(vd/2)):
  print("Uploading the input video...")
  video_path = f"/content/sample_data/TARGET/output{numvid}.mp4"  # Replace with your video path

  # Move the video file to the upload path
  shutil.move(video_path, os.path.join(upload_path, "input.mp4"))

  print("Video uploaded successfully.")

  # Define the path to the video file
  path = os.path.join(upload_path, "input.mp4")

  # Load the video clip
  clip = VideoFileClip(path)

  # Display the video clip
  clip.ipython_display(width=600)



  # Define the path to the reference image file
  image_path = f"/content/sample_data/REFS/source{numvid}.jpg"  # Replace with the actual image path

  # Set the filename for the reference image
  filename = "ref.jpg"

  # Move the reference image to the upload reference directory
  shutil.move(image_path, os.path.join(upload_ref_path, filename))

  # Display the reference image
  Image(os.path.join(upload_ref_path, filename), width=400)



  def FrameCapture(video_path, output_path):
      vidObj = cv2.VideoCapture(video_path)
      count = 0
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              cv2.imwrite(output_path + "/%d.jpg" % count, image)
              count += 1

  path = upload_path
  output_path = os.path.join(basepath, "frames")

  videos = [video for video in os.listdir(path) if video.endswith(".avi") or video.endswith(".mp4")]
  videos.sort()
  print(videos)

  print("extracting frames from input video")
  for idx, video in enumerate(videos):
      frame_path = os.path.join(output_path, video.split(".")[0])
      if not os.path.exists(frame_path):
          os.makedirs(frame_path)
      FrameCapture(os.path.join(path, video), frame_path)
  # Define the path to the image file

  image_path = './my_test/upload_ref/ref.jpg'

  # Read the image from the specified path
  image = cv2.imread(image_path)
  print(image.shape)


  !python test.py --clip_path ./my_test/frames/input \
               --ref_path ./my_test/upload_ref \
               --output_path ./my_test/upload_output

  path= os.path.join(basepath, "upload_output/input_ref/video.avi")
  clip=VideoFileClip(path)
  clip.ipython_display(width=600)
  import os

  old_path = os.path.join(basepath, "upload_output/input_ref/video.avi")
  new_path = os.path.join(basepath, f"upload_output/input_ref/video{numvid}.avi")

  os.rename(old_path, new_path)

In [ ]:
"""import os
from moviepy.editor import VideoFileClip, concatenate_videoclips

def concatenate_videos(folder_path, output_path):
    video_clips = []

    # Get a list of video files in the folder
    video_files = sorted([f for f in os.listdir(folder_path) if f.startswith('video') and f.endswith('.mp4')])

    # Load each video clip and add it to the list
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        clip = VideoFileClip(video_path)
        video_clips.append(clip)

    # Concatenate the video clips
    final_clip = concatenate_videoclips(video_clips)

    # Write the final concatenated video to the output path
    final_clip.write_videofile(output_path, codec='libx264')

    # Close the video clips
    final_clip.close()
    for clip in video_clips:
        clip.close()

# Example usage
folder_path = '/content/sample_data/TARGET'  # Replace with the actual folder path
output_path = '/content/sample_data/TARGET/outputbw.mp4'  # Replace with the desired output file path

concatenate_videos(folder_path, output_path)
"""

In [ ]:
import os
import shutil

source_folder = "/content/drive/MyDrive/Deep-Exemplar-based-Video-Colorization/my_test/upload_output/input_ref"
destination_folder = "/content/sample_data/Results"

# Iterate over all files in the source folder
for filename in os.listdir(source_folder):
    file_path = os.path.join(source_folder, filename)

    # Check if the file is a video file
    if os.path.isfile(file_path) and filename.lower().endswith(('.mp4', '.avi', '.mov')):
        # Move the video file to the destination folder
        shutil.move(file_path, os.path.join(destination_folder, filename))


In [ ]:
import os
import shutil
from moviepy.editor import VideoFileClip, concatenate_videoclips

def concatenate_videos(folder_path, output_path):
    # Get a list of video files in the folder
    video_files = [file for file in sorted(os.listdir(folder_path)) if file.lower().endswith(('.mp4', '.avi', '.mov'))]

    # Create an empty list to store the video clips
    video_clips = []

    # Iterate over the video files
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)

        # Load the video clip
        clip = VideoFileClip(video_path)
        video_clips.append(clip)

    # Concatenate the video clips sequentially
    final_clip = concatenate_videoclips(video_clips)
    final_clip = final_clip.resize(height=height,width=width)

    # Specify the output file path for the concatenated video
    output_file = os.path.join(output_path, "concatenated_video.avi")

    # Write the concatenated video to the output file
    final_clip.write_videofile(output_file, codec="mpeg4")

    print("Videos have been concatenated.")

# Specify the folder containing the video files
folder_path = "/content/sample_data/Results"

# Specify the output path for the concatenated video
output_path = "/content/sample_data/Results"

# Call the function to concatenate the videos
concatenate_videos(folder_path, output_path)
